In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [10]:
data_train = pd.read_csv("/Users/wuyijiong/Desktop/kaggle-Titanic-predict/Titanic_predict/data/train.csv")

In [11]:
#Age个Cabin有大量缺失
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


<font face="微软雅黑" size=4 color=#1E90F> 填补缺失值：
<font face="微软雅黑" size=2 color=black>
如果缺值的样本占总数比例极高，我们可能就直接舍弃了，作为特征加入的话，可能反倒带入noise，影响最后的结果了   
如果缺值的样本适中，而该属性非连续值特征属性(比如说类目属性)，那就把NaN作为一个新类别，加到类别特征中   
如果缺值的样本适中，而该属性为连续值特征属性，有时候我们会考虑给定一个step(比如这里的age，我们可以考虑每隔2/3岁为一个步长)，然后把它离散化，之后把NaN作为一个type加到属性类目中。   
有些情况下，缺失的值个数并不是特别多，那我们也可以试着根据已有的值，拟合一下数据，补充上。   



In [12]:
from sklearn.ensemble import RandomForestRegressor
### 使用 RandomForestClassifier 填补缺失的年龄属性
def set_missing_ages(df):
    
    # 把已有的数值型特征取出来丢进Random Forest Regressor中
    age_df = df[['Age','Fare', 'Parch', 'SibSp', 'Pclass']]

    # 乘客分成已知年龄和未知年龄两部分
    known_age = age_df[age_df.Age.notnull()].values
    unknown_age = age_df[age_df.Age.isnull()].values
    
    # y即目标年龄
    y = known_age[:, 0]

    # X即特征属性值
    X = known_age[:, 1:]

    # fit到RandomForestRegressor之中
    rfr = RandomForestRegressor(random_state=0, n_estimators=2000, n_jobs=-1)
    rfr.fit(X, y)
    
    # 用得到的模型进行未知年龄结果预测
    predictedAges = rfr.predict(unknown_age[:, 1:])
    
    # 用得到的预测结果填补原缺失数据
    df.loc[ (df.Age.isnull()), 'Age' ] = predictedAges 
    
    return df, rfr
#设置Cabin属性为是否有船舱号
def set_Cabin_type(df):
    df.loc[ (df.Cabin.notnull()), 'Cabin' ] = "Yes"
    df.loc[ (df.Cabin.isnull()), 'Cabin' ] = "No"
    return df

data_train, rfr = set_missing_ages(data_train)
data_train = set_Cabin_type(data_train)


<font face="微软雅黑" size=4 color=#1E90F> 分类变量构建虚拟变量(类似one-hot)
<font face="微软雅黑" size=2 color=black>
get_dummies和one-hot的区别https://stackoverflow.com/questions/48170405/is-pd-get-dummies-one-hot-encoding


In [13]:
#prefix：变成虚拟变量后列名的前缀
dummies_Cabin = pd.get_dummies(data_train['Cabin'], prefix= 'Cabin')

dummies_Embarked = pd.get_dummies(data_train['Embarked'], prefix= 'Embarked')

dummies_Sex = pd.get_dummies(data_train['Sex'], prefix= 'Sex')

dummies_Pclass = pd.get_dummies(data_train['Pclass'], prefix= 'Pclass')

df = pd.concat([data_train, dummies_Cabin, dummies_Embarked, dummies_Sex, dummies_Pclass], axis=1)
df.drop(['Pclass', 'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], axis=1, inplace=True)
df


,PassengerId,Survived,Age,SibSp,Parch,Fare,Cabin_No,Cabin_Yes,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3
0,1,0,22.000000,1,0,7.2500,1,0,0,0,1,0,1,0,0,1
1,2,1,38.000000,1,0,71.2833,0,1,1,0,0,1,0,1,0,0
2,3,1,26.000000,0,0,7.9250,1,0,0,0,1,1,0,0,0,1
3,4,1,35.000000,1,0,53.1000,0,1,0,0,1,1,0,1,0,0
4,5,0,35.000000,0,0,8.0500,1,0,0,0,1,0,1,0,0,1
5,6,0,23.838953,0,0,8.4583,1,0,0,1,0,0,1,0,0,1
6,7,0,54.000000,0,0,51.8625,0,1,0,0,1,0,1,1,0,0
7,8,0,2.000000,3,1,21.0750,1,0,0,0,1,0,1,0,0,1
8,9,1,27.000000,0,2,11.1333,1,0,0,0,1,1,0,0,0,1
9,10,1,14.000000,1,0,30.0708,1,0,1,0,0,1,0,0,1,0


<font face="微软雅黑" size=4 color=#1E90F> 数据归一化


In [14]:
#均值为0，标准差为1
import sklearn.preprocessing as preprocessing
scaler = preprocessing.StandardScaler()
age_scale_param = scaler.fit(df['Age'].values.reshape(-1,1))
df['Age_scaled'] = scaler.fit_transform(df['Age'].values.reshape(-1,1), age_scale_param)
fare_scale_param = scaler.fit(df['Fare'].values.reshape(-1,1))
df['Fare_scaled'] = scaler.fit_transform(df['Fare'].values.reshape(-1,1), fare_scale_param)


In [15]:
df.to_csv("/Users/wuyijiong/Desktop/kaggle-Titanic-predict/Titanic_predict/data/train_feature.csv")

<font face="微软雅黑" size=4 color=#1E90F> 测试集处理


In [35]:
data_test = pd.read_csv("/Users/wuyijiong/Desktop/kaggle-Titanic-predict/Titanic_predict/data/test.csv")

In [36]:
#填补一个缺失的fare
data_test.loc[(data_test.Fare.isnull()),'Fare']=0

In [37]:
#预测年龄
age_test = data_test[['Age','Fare', 'Parch', 'SibSp', 'Pclass']]
null_age = age_test[age_test.Age.isnull()].values
X_test = null_age[:, 1:]
predictedAges = rfr.predict(X_test)
data_test.loc[ (data_test.Age.isnull()), 'Age' ] = predictedAges

In [38]:
#构建特征是否有Cabin号
data_test = set_Cabin_type(data_test)

In [39]:
#分类属性进行one-hot
dummies_Cabin_test = pd.get_dummies(data_test['Cabin'],prefix='Cabin')
dummies_Embarked_test = pd.get_dummies(data_test['Embarked'],prefix='Embarked')
dummies_Sex_test = pd.get_dummies(data_test['Sex'],prefix='Sex')
dummies_Pclass_test = pd.get_dummies(data_test['Pclass'],prefix='Pclass')
test_feature = pd.concat([data_test,dummies_Cabin_test,dummies_Embarked_test,dummies_Pclass_test,dummies_Sex_test],axis=1)
test_feature.drop(['Pclass', 'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], axis=1, inplace=True)

In [46]:
test_feature['Age_scaled']=scaler.fit_transform(test_feature['Age'].values.reshape(-1,1),age_scale_param)
test_feature['Fare_scaled']=scaler.fit_transform(test_feature['Fare'].values.reshape(-1,1),age_scale_param)

In [48]:
test_feature.to_csv("/Users/wuyijiong/Desktop/kaggle-Titanic-predict/Titanic_predict/data/test_feature.csv")